In [1]:
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
from joblib import load
import matplotlib
matplotlib.use('TKAgg')
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
import plotly.graph_objects as go
import os
from inverse import *
from utils import create_dir

import sys
np.set_printoptions(threshold=sys.maxsize, edgeitems=30, linewidth=100000, precision=2)
from inverse import random_target_2d

EPSILON = 1e-6

# Auto target

In [54]:
# # 1. load your existing config
# with open("config_regression.json", "r") as f:
#     config = json.load(f)

# # 2. generate a new target
# target = random_target_2d(0, 3, 20.0, 50.0, 1.0, 3.0)

# # 3. replace stopband/passband
# config["stopband"] = target["stopband"]
# config["passband"] = target["passband"]

# # 4. write it back (or to a new file)
# # with open("config_regression.json", "w") as f:
# #     json.dump(config, f, indent=4)

# # print("Updated config_regression.json:")
# # print(json.dumps(config, indent=4))

# Read Config

In [55]:
# Load configuration
current_dir = os.getcwd() 
config_file_path = os.path.join(current_dir, "config_regression.json")
print(config_file_path)

config_file_path = config_file_path.replace("\\", "/")

with open(config_file_path, 'r') as file:
    config = json.load(file)

reg = load('results_rf_regression/forest.joblib')
des_var_bounds = np.load('results_rf_regression/des_var_bounds.npy')

# Define target bandgap
stopband = config['stopband']
passband = config['passband']
print('stopband:', stopband)
print('passband:', passband)

num_mode = config['num_mode']
num_k = config['num_k']
raw_data = pd.read_csv(config['data_path'], sep=',')

des_var_names = config['design_variable_names']
n_des_vars = len(des_var_names)

estimators = reg.estimators_
n_estimators = len(estimators)

d:\My Drive\Research\WAM\RIGID_2DAM_reg\config_regression.json
stopband: [[2, 3, 45.2, 46.21]]
passband: [[0, 3, 28.42, 30.15]]


## Plot dataset

In [56]:
import re
from matplotlib.lines import Line2D
import plotly.graph_objects as go
from bandgap_evaluator import check_feasibility_from_band_diagram
from inverse import calculate_proba_in_des_space_forward_regression, convert_target_format

In [57]:
## read band structure data from csv files

folder_path = r'D:\My Drive\Research\WAM\RIGID_2DAM_REG\dataset\lhs_consider_constrain1_n_samples_500_num_k_61_num_mode_15_seed_33'


designs = []
band_structures = []

for file_name in os.listdir(folder_path):
    match = re.match(r'result_rstr_([\d.]+)_rcenter_([\d.]+)_rcorner_([\d.]+)\.csv', file_name)
    if match:  # Check if file matches the required pattern
        rstr = float(match.group(1))
        rcenter = float(match.group(2))
        rcorner = float(match.group(3))
        designs.append({'rstr': rstr, 'rcenter': rcenter, 'rcorner': rcorner})
        
    file_path = os.path.join(folder_path, file_name)
    csv_data = pd.read_csv(file_path, skiprows = 4) 
    csv_data['Eigenfrequency (kHz)'] = csv_data['Eigenfrequency (kHz)'].apply(
        lambda x: complex(x.replace('i', 'j'))) 
    band_structures.append(csv_data)


In [58]:
num_mode = 15
num_k = 61
freq_range_of_interests = [0, 60]
threshold = 1

In [59]:
bg_type, target_freq_range, index_stopband = convert_target_format(stopband, passband, num_k)

In [60]:
obj_val_list = []
k_stride = (num_k - 1)/3.0

for i, design in enumerate(designs):
    data = band_structures[i]
    data['Frequency_real'] = data['Eigenfrequency (kHz)'].apply(lambda x: np.real(x))
    freq = data['Frequency_real'].values.reshape(num_k, num_mode)
    # obj_val = objective_function_arbitrary(data, num_mode, num_k, bg_type, 
    #                                        target_freq, index_stopband, threshold, freq_range_of_interests)
    obj_val = check_feasibility_from_band_diagram(freq, num_mode, num_k, bg_type, 
                                target_freq_range, index_stopband, threshold, freq_range_of_interests)
    obj_val_list.append({
        'design': design,
        'objective value': obj_val
    })

    freq = data['Frequency_real'].values.reshape(num_k, num_mode)
    k_points = data['% k'].values.reshape(num_k, num_mode)[:, 0]

In [65]:
feasible_indices = [i for i, item in sorted(enumerate(obj_val_list), 
                     key=lambda x: x[1]['objective value'], 
                     reverse=True) 
                  if item['objective value'] == 1]

print(f"Number of feasible designs: {len(feasible_indices)}")

Number of feasible designs: 40


In [66]:
property_distribution = []
property_distribution_remap = []

for item in obj_val_list:
    design = item['design']
    feasibility_val = item['objective value']
    property_distribution.append({
        'rstr': design['rstr'],
        'rcenter': design['rcenter'],
        'rcorner': design['rcorner'],
        'Feasibility Value': feasibility_val,
    })

    rstr_val = design['rstr']
    rcenter_val = design['rcenter']
    rcorner_val = design['rcorner']
    

df_property_distribution = pd.DataFrame(property_distribution)

min_size = 5
size_rate = 10

df_property_distribution['Point Size'] = np.where(
    df_property_distribution['Feasibility Value'] != 1,
    min_size,
    min_size + df_property_distribution['Feasibility Value'] * size_rate
)

fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=df_property_distribution['rstr'],
    y=df_property_distribution['rcenter'],
    z=df_property_distribution['rcorner'],
    mode='markers',
    marker=dict(
        size=df_property_distribution['Point Size'],
        color=np.where(
            df_property_distribution['Feasibility Value'] == 1,  # Condition for coloring
            'green',  # Color for Feasibility Value = 1
            'gray'  # Color for Feasibility Value != 1
        ),
        opacity=0.8,
        line=dict(width=1, color='DarkSlateGrey')
    ),
    name='Design Points',
    hoverinfo='text',
    hovertext='Feasibility Value: ' + df_property_distribution['Feasibility Value'].astype(str)
))


fig.update_layout(
    scene=dict(
        xaxis_title='<b>rstr (mm)</b>',
        yaxis_title='<b>rcenter (mm)</b>',
        zaxis_title='<b>rcorner (mm)</b>',
        xaxis=dict(range=[3.9, 6.5], gridcolor='lightgray'),
        yaxis=dict(range=[0, 20], gridcolor='lightgray'),
        zaxis=dict(range=[0, 20], gridcolor='lightgray'),
        bgcolor='rgba(245,245,245,0.8)'
    ),
    width=800,
    height=600)

fig.show() 

In [63]:
fig.write_html("scatter3d.html", include_plotlyjs="cdn")


## Compare probability distribution with dataset

In [64]:
stopband_str = "stopband_" + "_".join([f"k_{rng[0]}_{rng[1]}_freq_{rng[2]}_{rng[3]}" for rng in stopband])
passband_str = "passband_" + "_".join([f"k_{rng[0]}_{rng[1]}_freq_{rng[2]}_{rng[3]}" for rng in passband])

des_x, des_y, des_z, prob_forward = calculate_proba_in_des_space_forward_regression(des_var_bounds, passband, stopband, num_mode, num_k, reg, 10, 1)

In [67]:
fig = go.Figure(data=go.Volume(
    x=des_x,
    y=des_y,
    z=des_z,
    value = prob_forward,        
    isomin = prob_forward.min(),                # Minimum value to render
    isomax = prob_forward.max(),                 # Maximum value to render
    opacity = 0.6,                # Opacity (lower values = more transparent)
    opacityscale=[
    [0, 0],   # val = 0 ➝ opacity = 0 (transparent)
    [0.5, 0.25], # val = 0.5 ➝ opacity = 0.5
    [0.75, 0.5], # val = 0.75 ➝ opacity = 0.75
    [1, 1]    # val = 1 ➝ opacity = 1 (opaque)
    ],             # Opacity (lower values = more transparent)
    surface_count = 20,           # Number of transparent isosurfaces
    colorscale="Reds",       # Heatmap color scale (choose from Plotly's color scales)
        colorbar=dict(
        # title="Likelihood",
        title_font=dict(size=28),
        tickfont=dict(size=28),
        len=0.8,
    )
))

# Update layout
fig.update_layout(
    # title=f"{stopband_str}; {passband_str}",
    autosize=False,
    width=1000, 
    height=1000,
    margin=dict(l=65, r=50, b=65, t=90),
    scene=dict(
        xaxis_title=des_var_names[0],
        yaxis_title=des_var_names[1],
        zaxis_title=des_var_names[2],
        xaxis=dict(title_font=dict(size=28), tickfont=dict(size=18)),
        yaxis=dict(title_font=dict(size=28), tickfont=dict(size=18)),
        zaxis=dict(title_font=dict(size=28), tickfont=dict(size=18)),
    )
)
fig.show()

In [68]:
fig = go.Figure(data=go.Volume(
    x=des_x,
    y=des_y,
    z=des_z,
    value = prob_forward,        
    isomin = prob_forward.min(),                # Minimum value to render
    isomax = prob_forward.max(),                 # Maximum value to render
    opacity = 0.6,                # Opacity (lower values = more transparent)
    opacityscale=[
    [0, 0],   # val = 0 ➝ opacity = 0 (transparent)
    [0.5, 0.25], # val = 0.5 ➝ opacity = 0.5
    [0.75, 0.5], # val = 0.75 ➝ opacity = 0.75
    [1, 1]    # val = 1 ➝ opacity = 1 (opaque)
    ],             # Opacity (lower values = more transparent)
    surface_count = 20,           # Number of transparent isosurfaces
    colorscale="Reds",       # Heatmap color scale (choose from Plotly's color scales)
))

# fig.add_trace(go.Scatter3d(
#     x=df_property_distribution['rstr'],
#     y=df_property_distribution['rcenter'],
#     z=df_property_distribution['rcorner'],
#     mode='markers',
#     marker=dict(
#         size=df_property_distribution['Point Size'],
#         color=np.where(
#             df_property_distribution['Feasibility Value'] == 1,  # Condition for coloring
#             'green',  # Color for Feasibility Value = 1
#             'gray'  # Color for Feasibility Value != 1

#         ),
#         opacity=0.8,
#         line=dict(width=1, color='DarkSlateGrey')
#     ),
#     name='Design Points',
#     hoverinfo='text',
#     hovertext='Feasibility Value: ' + df_property_distribution['Feasibility Value'].astype(str)
# ))

# Update layout
fig.update_layout(
    title=f"{stopband_str}; {passband_str}",
    autosize=False,
    width=800, 
    height=800,
    margin=dict(l=65, r=50, b=65, t=90),
    scene=dict(
    xaxis_title=des_var_names[0],
    yaxis_title=des_var_names[1],
    zaxis_title=des_var_names[2]
))

# Save the plot
exp_dir = 'results_rf_regression/exp_' + stopband_str + passband_str
create_dir(exp_dir)
plot_path = '{}/design_space.html'.format(exp_dir)
fig.write_html(plot_path)

fig.show()

# Generate

In [69]:
# i = np.argmax(prob_forward)                              
# initial_design = np.array([des_x[i], des_y[i], des_z[i]])

# # Generate designs from the obtained design variable ranges
# print('Generating designs ...')
# n_designs = config['num_generated_designs'] # Number of designs to generate
# cat_var_indices = config['categorical_variable_indices']
# sample_threshold = config['sample_threshold']

# # designs, probabilities = generate(n_designs, list_des_var_ranges, des_var_bounds, cat_var_indices)
# n_designs = 20
# generated_designs, probabilities = generate_forward_regression(n_designs, passband, stopband, num_mode, num_k, des_var_bounds, reg, cat_var_indices, 1, initial_design, 0.2)
    

In [70]:
# if len(generated_designs) > 0:
    
#     # Create a folder for this experiment
#     exp_dir = 'results_rf_regression/exp_' + stopband_str + passband_str
#     create_dir(exp_dir)
    
#     # with open('{}/config.json'.format(exp_dir), 'w') as fout:
#     #     config['target_bandgap'] = target_bandgap
#     #     json.dump(config, fout)
    
#     # Save feasible design variable ranges for all the estimators
#     # with open('{}/list_des_var_ranges.json'.format(exp_dir), 'w') as fout:
#     #     json.dump(list_des_var_ranges_filtered, fout)

#     # Convert to serializable dictionaries
#     # serializable_meshes = [
#     #     mesh.to_dict() for mesh in list_des_var_ranges_filtered
#     # ]

#     # # Save to JSON file
#     # with open(f"{exp_dir}/list_des_var_ranges.json", "w") as fout:
#     #     json.dump(serializable_meshes, fout)
        
#     # Save designs and their probabilities
#     gen_des_df = pd.DataFrame(generated_designs, columns=des_var_names)
#     gen_des_df['Probability'] = probabilities
#     gen_des_path = '{}/generated_designs.csv'.format(exp_dir)
#     gen_des_df.to_csv(gen_des_path, index=False)
    


In [71]:
exp_dir = 'results_rf_regression/exp_' + stopband_str + passband_str
gen_des_path = '{}/generated_designs.csv'.format(exp_dir)

In [72]:
import pandas as pd

# Re‐construct the path
exp_dir       = 'results_rf_regression/Validation_v2/exp_' + stopband_str + passband_str
gen_des_path  = f'{exp_dir}/generated_designs.csv'

# Read it in
gen_des_df = pd.read_csv(gen_des_path)

# Recover the design‐variable array and the probability array
# as NumPy (or Python) arrays exactly as before:
generated_designs = gen_des_df[des_var_names].values    # shape (N, len(des_var_names))
probabilities     = gen_des_df['Probability'].values    # shape (N,)

# (If you prefer Python lists:)
# generated_designs = gen_des_df[des_var_names].values.tolist()
# probabilities     = gen_des_df['Probability'].values.tolist()


In [73]:
gen_des_df_sorted = gen_des_df.sort_values(by='Probability', ascending=False)

gen_des_df_sorted

,Strut Radius,Center Mass Radius,Corner Mass Radius,Probability
14,5.418340,18.909218,19.326637,1.00
16,4.907604,16.559201,19.687349,1.00
11,5.108707,18.305423,19.966065,1.00
9,5.099680,17.210297,19.107721,0.98
10,4.562302,18.866096,19.784257,0.91
17,4.899302,15.169590,19.212130,0.80
8,6.391082,18.282300,18.169672,0.74
12,4.583200,18.094511,11.143047,0.65
15,4.136802,19.866531,18.738262,0.60
1,6.271609,18.879220,17.942459,0.58


In [74]:
probabilities

array([0.23, 0.58, 0.55, 0.58, 0.41, 0.16, 0.01, 0.57, 0.74, 0.98, 0.91, 1.  , 0.65, 0.26, 1.  , 0.6 , 1.  , 0.8 , 0.08, 0.48])

In [76]:
fig = go.Figure(data=go.Volume(
    x=des_x,
    y=des_y,
    z=des_z,
    value = prob_forward,        
    isomin = prob_forward.min(),                # Minimum value to render
    isomax = prob_forward.max(),                 # Maximum value to render
    opacity = 0.6,                # Opacity (lower values = more transparent)
    opacityscale=[
    [0, 0],   # val = 0 ➝ opacity = 0 (transparent)
    [0.5, 0.25], # val = 0.5 ➝ opacity = 0.5
    [0.75, 0.5], # val = 0.75 ➝ opacity = 0.75
    [1, 1]    # val = 1 ➝ opacity = 1 (opaque)
    ],             # Opacity (lower values = more transparent)
    surface_count = 20,           # Number of transparent isosurfaces
    colorscale="Reds",       # Heatmap color scale (choose from Plotly's color scales)
    colorbar=dict(
        title="Likelihood",
        title_font=dict(size=28),
        tickfont=dict(size=28),)
))

fig.add_trace(go.Scatter3d(
    x=gen_des_df["Strut Radius"], 
    y=gen_des_df["Center Mass Radius"],
    z=gen_des_df["Corner Mass Radius"], 
    mode='markers', 
    marker=dict(
        # size=gen_des_df['Probability'] * 30,  # Adjust size based on probability
        size=5,
        color='blue',  # Color for generated designs
        opacity=0.5),
        showlegend=False))

# fig.add_trace(go.Scatter3d(
#     x=df_property_distribution['rstr'],
#     y=df_property_distribution['rcenter'],
#     z=df_property_distribution['rcorner'],
#     mode='markers',
#     marker=dict(
#         size=df_property_distribution['Point Size'],
#         color=np.where(
#             df_property_distribution['Objective Value'] == 0,  # Condition for coloring
#             'gray',  # Color for Objective Value = 0
#             'green'  # Color for Objective Value != 0
#         ),
#         opacity=0.8,
#         line=dict(width=1, color='DarkSlateGrey')
#     ),
#     name='',
#     hoverinfo='text',
#     hovertext='Objective Value: ' + df_property_distribution['Objective Value'].astype(str)
# ))

# Update layout
fig.update_layout(
    title=f"{stopband_str}; {passband_str}",
    autosize=False,
    width=800, 
    height=800,
    margin=dict(l=65, r=50, b=65, t=90),
    scene=dict(
    xaxis_title=des_var_names[0],
    yaxis_title=des_var_names[1],
    zaxis_title=des_var_names[2],
    xaxis=dict(title_font=dict(size=20), tickfont=dict(size=16)),
    yaxis=dict(title_font=dict(size=20), tickfont=dict(size=16)),
    zaxis=dict(title_font=dict(size=20), tickfont=dict(size=16))
))

# Save the plot
# plot_path = '{}/design_space.html'.format(exp_dir)
# fig.write_html(plot_path)

fig.show()

In [109]:
from pathlib import Path

exp_dir = Path(exp_dir)

output_path = exp_dir / "generate3d.html"

fig.write_html(str(output_path), include_plotlyjs="cdn")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\My Drive\\Research\\WAM\\wam\\dataset\\validation_2'

<Figure size 432x576 with 0 Axes>